# Etude de cas de l'utilisation des IA Génératives (LLMs) dans l'enseignement supérieur de la science du numérique.

---

## Préambule

Les IA génératives telles que ChatGPT, DeepSeek, ou Claude Sonnet sont indéniablement devenues ces dernières années des outils très prominents dans le quotidien numérique des étudiants. Étant des outils très puissants, il est essentiel de savoir s'en servir convenablement pour en tirer le maximum. Cela demande en premier lieu d'*au moins* comprendre dans les très (très) grandes lignes leur principe de fonctionnement et ce que la technologie implique, mais aussi leurs potentiel et limites actuelles. Cela évitera notamment ce genre de remarques:

<div>
    <center>
    <img src="img/illustrative_tweet.png" width="500"/>
</div>

## Principe de l'IA Générative

Pour faire très (très) simple en omettant toutes les maths derrière:

&rarr; un **modèle de langage** comme **GPT 4.1** (≠ ChatGPT!) va, à partir d'un texte en entrée (un contexte en fait), découper ce texte en morceaux/~mots appelés *tokens* et leur attribuer un vecteur dans un espace de très grande dimension (beaucoup d'axes). 

&rarr; Ce vecteur déterminera d'une certaine manière le sens "*sémantique*" d'un token, et ainsi les autres mots de sens similaires auront des vecteurs proches dans l'espace. Les valeurs qui caractérisent ces vecteurs sont choisies aléatoirement puis apprises lors de l'**apprentissage** du modèle; une étape antérieure où on apprend au modèle à comparer et donc catégoriser les entrées qu'on lui donne.

Plus bas il y a par exemple un code en Python que vous pouvez exécuter (pas besoin de chercher à le comprendre, il faut juste cliquer sur le bouton latéral gauche pour démarrer la zone de code) afin d'observer l'angle entre des vecteurs de tokens:

In [15]:
## On installe une librairie qui permet de télécharger de larges modèles utilisés dans le traîtement du 
# langage naturel

#à ne lancer qu'une fois et à installer sur Python <=3.12 !
pip install gensim

SyntaxError: invalid syntax (1723020062.py, line 5)

On utilise le modèle ***glove-wiki-gigaword-50***, qui n'est pas un LLM mais un *Word Embedding Model*. Comme son nom l'indique peut-être, il y a 50 dimensions dans un vecteur; Sachez par ailleurs que pour GPT-3, il y en a 12 288. Il s'agit dans tous les cas exactement de ce qu'on a décrit dans le paragraphe précédent: C'est un modèle qui capture les relations sémantiques entre les mots et peut les représenter dans des vecteurs spaciaux qui symbolisent leur proximité sémantique.

On importe *gensim* et on télécharge le modèle:

In [ ]:
import gensim.downloader # On importe la librairie qui nous permet de télécharger des modèles de 
# plongement de mots

model = gensim.downloader.load("glove-wiki-gigaword-50")

On va observer la proximité sémantique des tokens en regardant l'angle que font leur vecteur:

In [ ]:
# On va utiliser les fonctions intégrées à Numpy pour traiter nos vecteurs
import numpy as np

## Fonctions utilitaires
def normaliser(vecteur):
    """ Normalise le vecteur en argument.
    """
    return np.round(vecteur / np.linalg.norm(vecteur), 3) # arrondi au milième

def angle(v1, v2):
    """Retourne l'angle fait par les vecteurs donnés en argument.
    """
    v1_u = normaliser(v1)
    v2_u = normaliser(v2)
    res = np.dot(v1_u, v2_u)
    res = np.clip(res, -1.0, 1.0)
    res = np.arccos(res)
    res *= 180/np.pi
    return round(res, 3)

## Affichage des vecteurs

#Exemple avec les vecteurs des mots "tour" et "bâtiment" 
vecteur_tour = model["tower"] 

vecteur_building = model["building"]

print(f"Le vecteur normalisé du mot 'tower' est:\n {normaliser(vecteur_tour)}\n")
print(f"Le vecteur normalisé du mot 'building' est:\n {normaliser(vecteur_building)}\n")
print(f"L'angle entre les deux vecteurs en degrés est environ {angle(vecteur_tour, vecteur_building)}")



Le vecteur normalisé du mot 'tower' est:
 [ 0.232  0.239  0.151 -0.012  0.102 -0.143 -0.065 -0.092 -0.093 -0.153
 -0.068 -0.005 -0.102  0.129 -0.168  0.169 -0.05   0.066 -0.225 -0.004
  0.139 -0.08  -0.332 -0.102 -0.034 -0.137 -0.064  0.179 -0.006 -0.032
  0.401 -0.24   0.169 -0.037 -0.054  0.024  0.223 -0.073  0.005 -0.008
  0.062 -0.083  0.017  0.046 -0.103  0.133 -0.025 -0.284  0.    -0.106]

Le vecteur normalisé du mot 'building' est:
 [ 0.177  0.095  0.075 -0.081  0.096 -0.08  -0.162 -0.132 -0.06  -0.109
 -0.113 -0.106 -0.105  0.01  -0.105  0.089  0.019  0.148  0.013 -0.046
  0.235  0.03  -0.301 -0.134 -0.074 -0.197 -0.01   0.078  0.054  0.007
  0.614 -0.175  0.031 -0.162 -0.013  0.118  0.123  0.079  0.076  0.079
 -0.07  -0.187  0.027  0.009 -0.014  0.099 -0.146 -0.135 -0.005 -0.054]

L'angle entre les deux vecteurs en degrés est environ 38.057


Voici un autre exemple avec des mots sans grande liaison sémantique:

In [ ]:

vecteur_printer = model["printer"] # veccteur pour imprimante

vecteur_platypus = model["platypus"] # vecteur pour ornythorinque

# print(f"Le vecteur normalisé du mot 'game' est:\n {normaliser(vecteur_tour)}\n")
# print(f"Le vecteur normalisé du mot 'player' est:\n {normaliser(vecteur_building)}\n")
print(f"L'angle entre les deux vecteurs en degrés est environ {angle(vecteur_printer, vecteur_platypus)}")


L'angle entre les deux vecteurs en degrés est environ 105.672



&rarr; Ces tokens échangent ensuite entre eux pour décider quels éléments du contexte influent sur la **sémantique appropriée** de certains mots de la phrase. Par exemple, quand on dit "graver en mémoire", c'est un verbe 'graver' qui diffère de celui dans "graver dans de la pierre". On peut appeler cette étape "*bloc d'Attention*".

&rarr; Ensuite, ces vecteurs passent par une étape au nom barbare de *perceptron multi-couches* ... le procédé mathématique exact est un peu compliqué à vulgariser donc on va juste dire que c'est une manière pour le modèle de catégoriser et mettre à jour chaque vecteur de token, comme s'il lui posait à la suite pleins de questions et qu'il prenait en compte les réponses pour affiner le vecteur.

<div>
    <center>
    <img src="img/mlp.png" width="500"/>
</div>

&rarr; La suite est une répétition d'alternances entre **blocs d'attention** et **perceptrons multi-couches**, jusqu'à la dernière itération où le dernier vecteur de la séquence sera donc le résultat combiné de tout ce passage.

&rarr; En opérant sur ce vecteur, on obtient une **distribution de probabilités** de tous les *tokens* possibles en sortie (donc tous les mots/segments de texte qui peuvent se placer linguistiquement dans la continuité de notre texte initial). En gros, on obtient une liste des mots qui pourront **probablement** le mieux compléter la suite du texte donné.

Mais tout ¢a, c'est pour obtenir **un** mot. Ainsi, on peut recommencer une fois de plus en ajoutant notre mot nouvellement obtenu dans le contexte de l'itération suivante afin d'obtenir la suite. Et on peut recommencer encore et encore...

Enfin, pour obtenir un ***Chatbot*** (pour faire simple, ChatGPT et équivalent) depuis ce modèle, en fait c'est tout bête voire assez marrant: On appose, en préfixe du prompt de l'utilisateur, un *contexte caché* où on précise que ce qui suit est une discussion entre une IA et son utilisateur. Donc, le prompt de l'utilisateur sera donné à l'IA sous la forme du début d'un dialogue. De là, le chatbot va en fait prédire mot par mot la réponse la plus probable que ferait un assistant IA dans le dialogue.


## Limites des LLMs



-> L'une des limitations qu'on n'a pas abordées dans la section précédente est la **taille du contexte**: Il s'agit du nombre fixe de vecteurs (donc de tokens) que le réseau de neurones peut traîter. En pratique, c'est donc la taille maximale du texte que le modèle peut utiliser pour ses prédictions de mots. C'est à cause de cette taille de contexte que les divers Chatbots perdent parfois le fil lors de très longues conversations.

-> Une IA **Générative** n'est pas **Cŕeative**. Elle ***ne peut pas*** produire de contenu original et creátif ex-nihilo, mais dépend entièrement des données d'apprentissage qui lui ont été fournies. Ces données crystallisent, pour beaucoup, les **biais/disparités/préjudices/limitations** *historiquement ancrés* dans nos sociétés et ses écrits. Le risque que présente l'existence de ces biais dans la génération de texte d'une IA est l'empêchement pour la majorité de sortir des carcans de la normativité ou d'un mode de pensée conventionel, voire la perpétuation de traitements inégaux envers des groupes marginalisés (par le genre, l'éthnie, l'origine sociale, l'orientation sexuelle, etc.). En clair, [intégrer les LLMs dans des domaines où ces biais ont un impact direct sur le traitement de différentes populations pourrait amplifier et renforcer des biais historiques.](https://arxiv.org/html/2411.10915v1) (on vous invite fortement à lire cet article qui entre en très grand détail sur tous les biais des LLMs actuels et leur impact majeur dans l'interprétation/la génération de textes).

Évidemment, à échelle moindre, cela vaut aussi pour la production de contenu (comme du code): l'IA est entraînée sur une quantité massive de données parfois très similaires, et va finir par intégrer dans son style d'écriture / de code des tics de langage ou les tournures les plus habituelles de ses sources. Pour vous illustrer ¢a, un indice assez commun que les professeurs de lettres américains connaissent bien est le fait que ChatGPT en anglais a tendance à énormément abuser des tournures avec "*em-dash*" (le symbole "––") en fin de phrase–– Comme ceci par exemple. En clair, une IA **ne crée pas**; elle génère, de ce qu'elle a appris, le contenu  **le plus probable**.

Le biais le plus évident est aussi l'origine géographique/culturelle des données: Ces données sont très américano-centrées; Cela inclut notations, nomenclatures et terminologies à fortes consonnances nord-américaines.


-> Aussi, il faut rappeler que ces IAs sont entrainées pour produire une réponse qui *plait* à l'utilisateur: [Elles viennent avec un **biais de confirmation**](https://mediate.com/ai-and-confirmation-bias/) très impactant, qui les mène à aller *systématiquement* dans votre sens. Cela veint de l'*Apprentissage par Renforcement à partir du Feedback Humain*, où l'apprentissage se fait avec des individus qui valident ou non les réponses faites par les IAs et où on a observé que les participants avaient une forte tendance à mieux évaluer les productions qui allaient dans leur sens.

-> Enfin, à cela s'ajoute la limitation de base que sont les **hallucinations**: il s'agit de réponses que l'IA produit et qui sont totalement inattendues. On peut y inclure les inventions comme les pertes de contexte; Le problème étant que ces modèles sont avant tout faits pour reproduire le langage humain et paraissent donc constamment crédible même en pleine hallucination...

## Observer les pondérations que l'IA donne aux tokens

Pour observer la pondération que l'IA attribue aux tokens pour comprendre le contexte, on peut utiliser l'outil [*interpreto*](https://pypi.org/project/interpreto/), qui a pour ambition de rendre les LLMs plus transparents et de trouver une manière de mesurer leur fiabilité.

In [ ]:
pip install interpreto